# WeatherPy Analysis

## Question 
The fundamental question that we are trying to answer in this exercise is:
"What's the weather like as we approach the equator?"

## Objective
To visualize the weather of 500+ randomly selected cities across the world of varying distance from the equator. To build a series of scatter plots to showcase the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

## Technical Considerations
* We will use python's library - [random](https://docs.python.org/3/library/random.html#), to generate the random city coordinates to be used. This will ensure that the coordinates are in fact not manually picked, but generated randomly
* We will use a python library - [citipy](https://pypi.python.org/pypi/citipy), to get our list of cities from the randomly generated coordinates. This library will give us the nearest city for a given set of coordinates.
* We will use the [OpenWeatherMap API](https://openweathermap.org/api) to get the weather data for all the cities that we get from our random coordinates
* We will use Matplotlib and Seaborn libraries for Data Visualization

In [26]:
# Import Dependencies
from citipy import citipy
import random
import requests as req
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

In [16]:
# The API key for the OpenWeatherMap Api
api_key = "7e4ae0373bdeed043f36aeafca9fc7ff"

# The endpoint url for the api call to the OpenWeatherMap api
endpoint_url = "http://api.openweathermap.org/data/2.5/weather?q="

In [12]:
# Generate 130 random positive latitude numbers, negative latitude numbers, positive longitude numbers 
# and negative longitude numbers. This will ensure that we have coordinate samples from all four quadrants 
# of the globe. Furthermore we get random latitudes from two sections 0 to 45 and 45 to 90 to ensure even distribution
# of latitudes close to and away from the equator
pos_lats = [random.uniform(0, 45) for x in range(65)]
pos_lats += [random.uniform(45, 90) for x in range(65)]

neg_lats = [random.uniform(0, -45) for x in range(65)]
neg_lats += [random.uniform(-45, -90) for x in range(65)]

pos_lons = [random.uniform(0, 180) for x in range(130)]
neg_lons = [random.uniform(0, -180) for x in range(130)]

# List of coordinates for our cities
coords = []

# Create a combination of positive and negative latitides and longitudes to get our city coordinates
# This will give us a list of (130 * 4) = 520 coordinates
for i in range(130):
    coords.append((pos_lats[i], pos_lons[i]))
    coords.append((pos_lats[i], neg_lons[i]))
    coords.append((neg_lats[i], pos_lons[i]))
    coords.append((neg_lats[i], neg_lons[i]))

# Use the citipy library to get the nearest city for each of the set of coordinates generated above. 
# This should give us a list of 520 random cities along with their country codes
cities = [citipy.nearest_city(coord[0], coord[1]) for coord in coords]

print([(city.city_name, city.country_code) for city in cities])

[('marabba', 'sd'), ('sao filipe', 'cv'), ('sikonge', 'tz'), ('cuite', 'br'), ('butaritari', 'ki'), ('samana', 'do'), ('ahipara', 'nz'), ('san rafael', 'ar'), ('namatanai', 'pg'), ('itarema', 'br'), ('gizo', 'sb'), ('ibimirim', 'br'), ('kutum', 'sd'), ('les cayes', 'ht'), ('bloemfontein', 'za'), ('coquimbo', 'cl'), ('eyl', 'so'), ('puerto ayacucho', 've'), ('victoria', 'sc'), ('maraa', 'br'), ('oum hadjer', 'td'), ('kapaa', 'us'), ('luena', 'ao'), ('samusu', 'ws'), ('umm lajj', 'sa'), ('taoudenni', 'ml'), ('kijabe', 'ke'), ('takoradi', 'gh'), ('jijiga', 'et'), ('champerico', 'gt'), ('chake chake', 'tz'), ('puerto ayora', 'ec'), ('zhangye', 'cn'), ('ribeira grande', 'pt'), ('bengkulu', 'id'), ('aracati', 'br'), ('eydhafushi', 'mv'), ('wageningen', 'sr'), ('hithadhoo', 'mv'), ('itaituba', 'br'), ('ngama', 'td'), ('dinguiraye', 'gn'), ('hermanus', 'za'), ('jamestown', 'sh'), ('fanai', 'gr'), ('fortuna', 'us'), ('manono', 'cd'), ('atuona', 'pf'), ('that phanom', 'th'), ('guerrero negro', '

In [15]:
# Create a city table with the city names and country codes that we have gathered above. We will add the 
# other weather information for these cities later
cities_df = pd.DataFrame(data = {"City Name": [city.city_name for city in cities],
                                "Country Code": [ city.country_code for city in cities]})

In [19]:
# Add columns for the remaining data for all the cities that we will collect
cities_df = pd.concat([cities_df,pd.DataFrame(columns=["City ID", "Lat", "Lon", "Temperature", "Pressure", "Humidity", "Cloudiness", "Wind Speed"])])

# Reorder the columns
cities_df = cities_df[["City ID", "City Name", "Country Code", "Lat", "Lon", "Temperature", "Pressure", "Humidity", "Cloudiness", "Wind Speed"]]

In [20]:
cities_df.head()

,City ID,City Name,Country Code,Lat,Lon,Temperature,Pressure,Humidity,Cloudiness,Wind Speed
0,NaN,marabba,sd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,sao filipe,cv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,sikonge,tz,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,cuite,br,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,butaritari,ki,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def get_city_weather(index):
    city_str = cities_df.at[index, "City Name"] + "," + cities_df.at[index, "Country Code"]
    target_url = endpoint_url + city_str + "&units=imperial&appid=" + api_key
    
    print("Getting Data for %s with target_url = %s" %(city_str, target_url))
    city_response = req.get(target_url).json()

    # Get the information in a try block to ignore any cities that don't have all the information 
    try:
        cities_df.at[index, "City ID"] = city_response["id"]
        cities_df.at[index, "Lat"] = city_response["coord"]["lat"]
        cities_df.at[index, "Lon"] = city_response["coord"]["lon"]
        cities_df.at[index, "Temperature"] = city_response["main"]["temp"]
        cities_df.at[index, "Pressure"] = city_response["main"]["pressure"]
        cities_df.at[index, "Humidity"] = city_response["main"]["humidity"]
        cities_df.at[index, "Cloudiness"] = city_response["clouds"]["all"]
        cities_df.at[index, "Wind Speed"] = city_response["wind"]["speed"]
    except KeyError:
        print("Ignoring row due to Key Error for " + city_str)



In [40]:
for n in range(0,9):
    if (n == 8):
        print("Processing rows from " + str(n*60)+" to 520")
        #Process the last set of cities. These are less than 60
        for row_index in range(n*60,520):
            get_city_weather(row_index)
    else:
        for row_index in range(n*60,(n*60)+60):
            get_city_weather(row_index)

        # Introduce a sleep time for just a minute for every 60th row because the OpenWeatherMap api has
        # a limit of no more than 60 requests in a minute.
        print("Wait for a minute. We've got data upto city index number : " + str(row_index))
        time.sleep(60)

Getting Data for marabba,sd with target_url = http://api.openweathermap.org/data/2.5/weather?q=marabba,sd&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for sao filipe,cv with target_url = http://api.openweathermap.org/data/2.5/weather?q=sao filipe,cv&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for sikonge,tz with target_url = http://api.openweathermap.org/data/2.5/weather?q=sikonge,tz&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for cuite,br with target_url = http://api.openweathermap.org/data/2.5/weather?q=cuite,br&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for butaritari,ki with target_url = http://api.openweathermap.org/data/2.5/weather?q=butaritari,ki&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for samana,do with target_url = http://api.openweathermap.org/data/2.5/weather?q=samana,do&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for 

Getting Data for busselton,au with target_url = http://api.openweathermap.org/data/2.5/weather?q=busselton,au&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for rikitea,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=rikitea,pf&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for butaritari,ki with target_url = http://api.openweathermap.org/data/2.5/weather?q=butaritari,ki&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for kapaa,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=kapaa,us&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for kirakira,sb with target_url = http://api.openweathermap.org/data/2.5/weather?q=kirakira,sb&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for saleaula,ws with target_url = http://api.openweathermap.org/data/2.5/weather?q=saleaula,ws&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error 

Getting Data for viligili,mv with target_url = http://api.openweathermap.org/data/2.5/weather?q=viligili,mv&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for viligili,mv
Getting Data for san patricio,mx with target_url = http://api.openweathermap.org/data/2.5/weather?q=san patricio,mx&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bambous virieux,mu with target_url = http://api.openweathermap.org/data/2.5/weather?q=bambous virieux,mu&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for rikitea,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=rikitea,pf&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bilma,ne with target_url = http://api.openweathermap.org/data/2.5/weather?q=bilma,ne&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for lompoc,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=lompoc,us&units=imperial&appid=7e4ae

Getting Data for chesapeake,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=chesapeake,us&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for saint-philippe,re with target_url = http://api.openweathermap.org/data/2.5/weather?q=saint-philippe,re&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for ancud,cl with target_url = http://api.openweathermap.org/data/2.5/weather?q=ancud,cl&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for maridi,sd with target_url = http://api.openweathermap.org/data/2.5/weather?q=maridi,sd&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for maridi,sd
Getting Data for atuona,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=atuona,pf&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for standerton,za with target_url = http://api.openweathermap.org/data/2.5/weather?q=standerton,za&units=imperial&appid=7e4ae0373bd

Getting Data for hithadhoo,mv with target_url = http://api.openweathermap.org/data/2.5/weather?q=hithadhoo,mv&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for tefe,br with target_url = http://api.openweathermap.org/data/2.5/weather?q=tefe,br&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for port blair,in with target_url = http://api.openweathermap.org/data/2.5/weather?q=port blair,in&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for lazaro cardenas,mx with target_url = http://api.openweathermap.org/data/2.5/weather?q=lazaro cardenas,mx&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bengkulu,id with target_url = http://api.openweathermap.org/data/2.5/weather?q=bengkulu,id&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for bengkulu,id
Getting Data for rikitea,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=rikitea,pf&units=imperial&appid=7e4ae

...

Getting Data for evensk,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=evensk,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bethel,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=bethel,us&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bluff,nz with target_url = http://api.openweathermap.org/data/2.5/weather?q=bluff,nz&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for mataura,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=mataura,pf&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for mataura,pf
Wait for a minute. We've got data upto city index number : 299
Getting Data for talnakh,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=talnakh,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for ilulissat,gl with target_url = http://api.openweathermap.org/data/2.5/weather?q=ilu

Getting Data for saskylakh,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=saskylakh,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for illoqqortoormiut,gl with target_url = http://api.openweathermap.org/data/2.5/weather?q=illoqqortoormiut,gl&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for illoqqortoormiut,gl
Getting Data for albany,au with target_url = http://api.openweathermap.org/data/2.5/weather?q=albany,au&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for mar del plata,ar with target_url = http://api.openweathermap.org/data/2.5/weather?q=mar del plata,ar&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for deputatskiy,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=deputatskiy,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for bolungarvik,is with target_url = http://api.openweathermap.org/data/2.5/weather?q=bolunga

Getting Data for nizhneyansk,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=nizhneyansk,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for nizhneyansk,ru
Getting Data for barrow,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=barrow,us&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for hobart,au with target_url = http://api.openweathermap.org/data/2.5/weather?q=hobart,au&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for mataura,pf with target_url = http://api.openweathermap.org/data/2.5/weather?q=mataura,pf&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for mataura,pf
Getting Data for dikson,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=dikson,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for barrow,us with target_url = http://api.openweathermap.org/data/2.5/weather?q=barrow,us&

Getting Data for tumannyy,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=tumannyy,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Ignoring row due to Key Error for tumannyy,ru
Getting Data for narsaq,gl with target_url = http://api.openweathermap.org/data/2.5/weather?q=narsaq,gl&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for port elizabeth,za with target_url = http://api.openweathermap.org/data/2.5/weather?q=port elizabeth,za&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for ushuaia,ar with target_url = http://api.openweathermap.org/data/2.5/weather?q=ushuaia,ar&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for batagay,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=batagay,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for pangnirtung,ca with target_url = http://api.openweathermap.org/data/2.5/weather?q=pangnirtung,ca&units=imperial&appid=7e4ae

Getting Data for khatanga,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=khatanga,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for tasiilaq,gl with target_url = http://api.openweathermap.org/data/2.5/weather?q=tasiilaq,gl&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for busselton,au with target_url = http://api.openweathermap.org/data/2.5/weather?q=busselton,au&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for cidreira,br with target_url = http://api.openweathermap.org/data/2.5/weather?q=cidreira,br&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for cherskiy,ru with target_url = http://api.openweathermap.org/data/2.5/weather?q=cherskiy,ru&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for tuktoyaktuk,ca with target_url = http://api.openweathermap.org/data/2.5/weather?q=tuktoyaktuk,ca&units=imperial&appid=7e4ae0373bdeed043f36aeafca9fc7ff
Getting Data for blu

In [41]:
cities_df.count()

City ID         450
City Name       520
Country Code    520
Lat             450
Lon             450
Temperature     450
Pressure        450
Humidity        450
Cloudiness      450
Wind Speed      450
dtype: int64